In [1]:
import sys
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
import os
import cv2
import numpy as np

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf

sys.path.append('Datasets')
from StanfordDataset import StanfordDataset

sys.path.append('config')
import settings

sys.path.append('Preprocessings')
from StanfordPreprocessing import StanfordPreprocessing

sys.path.append('Models')
from InceptionV3 import InceptionV3

In [3]:
def read_labels(path):
    with open(path, 'r') as file:
        data = file.read()
    
    return np.array(data.split('\n'))

preprocessor = StanfordPreprocessing(read_labels(settings.LABELS_PATH), settings.IMAGE_HEIGHT, settings.IMAGE_WIDTH)

In [3]:
model = tf.keras.models.load_model("Artifacts/Models/" + desired_value + "/v1/Model/tf")
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 150, 150, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 150, 150, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
img = preprocessor.process_single_img('Data/StanfordDataset/images/Images/n02089867-Walker_hound/n02089867_90.jpg')
print(model.predict(tf.expand_dims(img, axis=0)))

1/1 [==============================] - 1s 1s/step
[array([[0.00714117]], dtype=float32), array([[0.01104495]], dtype=float32)]


In [37]:
data_ds = tf.data.Dataset.list_files('Data/StanfordDataset/images/Images/n02085620-Chihuahua/*')
data_ds = data_ds.map(preprocessor.process_single_img).batch(1)
result = model.predict(data_ds)

152/152 [==============================] - 13s 88ms/step


In [33]:
data_ds = tf.data.Dataset.list_files('Data/StanfordDataset/images/Images/n02091831-Saluki/*')
data_ds = data_ds.map(preprocessor.process_single_img).batch(1)
result = model.predict(data_ds)

200/200 [==============================] - 21s 104ms/step


In [39]:
for el in result[0]:
    # if(el > 0.2):
    print(el)

[0.01907595]
[0.01745297]
[0.03229853]
[0.02966036]
[4.922882e-09]
[0.00731059]
[0.00266702]
[0.00525521]
[0.00086013]
[0.02538346]
[0.02317095]
[0.03196611]
[0.01494391]
[0.02139817]
[0.03120237]
[0.0139052]
[0.01888439]
[0.01692764]
[0.02476757]
[0.02229314]
[0.01849787]
[0.02091635]
[0.01082548]
[0.02124209]
[0.00590601]
[0.00946184]
[0.03085693]
[0.00141565]
[0.0175022]
[0.02389925]
[0.01160532]
[0.02699912]
[0.01772471]
[0.02129362]
[0.02887673]
[0.03351799]
[0.02416471]
[0.02445765]
[0.0159399]
[0.0175881]
[0.00966185]
[0.02770131]
[0.03575999]
[0.02379108]
[0.03592438]
[0.02195068]
[0.03519451]
[0.01358914]
[0.03032331]
[0.00633251]
[0.01868437]
[0.02559857]
[0.02617912]
[0.00992413]
[0.0252322]
[0.00733891]
[0.02188919]
[0.03056436]
[0.02290729]
[0.02837905]
[0.01166329]
[0.01071276]
[0.0251267]
[0.0120334]
[0.02803132]
[1.6298226e-07]
[0.00194622]
[0.0270151]
[0.00641017]
[0.03460427]
[0.02067184]
[0.02538756]
[0.03310597]
[0.00428432]
[0.0321404]
[0.02683051]
[0.01322792]
[0.

In [3]:
stanfordDataset = StanfordDataset(
    data_path=settings.DATA_PATH,
    label_path=settings.LABELS_PATH,
    batch_size=1
)

In [4]:
(train_ds, val_ds, test_ds) = stanfordDataset.create_data_pipelines(preprocessor)

In [7]:
count = 0
for sample, label in train_ds:
    if(label.numpy()[0] != 1):
        continue
    count += 1
    # print(label)
    # print(model.predict(sample))
    # print('-'*100)
    # sample = (sample.numpy().reshape(299, 299, 3).shape)
    # cv2.imshow("img", sample)
    # cv2.waitKey(0)

print(count)

tf.Tensor([1], shape=(1,), dtype=int32)
1/1 [==============================] - 2s 2s/step
[array([[0.02731329]], dtype=float32), array([[0.01078565]], dtype=float32)]
----------------------------------------------------------------------------------------------------
tf.Tensor([1], shape=(1,), dtype=int32)
1/1 [==============================] - 0s 98ms/step
[array([[0.0321404]], dtype=float32), array([[0.01202576]], dtype=float32)]
----------------------------------------------------------------------------------------------------
tf.Tensor([1], shape=(1,), dtype=int32)
1/1 [==============================] - 0s 96ms/step
[array([[0.03229853]], dtype=float32), array([[0.02003356]], dtype=float32)]
----------------------------------------------------------------------------------------------------
tf.Tensor([1], shape=(1,), dtype=int32)
1/1 [==============================] - 0s 98ms/step
[array([[0.03005883]], dtype=float32), array([[0.01845662]], dtype=float32)]
------------------------

In [42]:
print(preprocessor.get_one_hot_label("Data/StanfordDataset/images/Images/n02085620-Chihuahua/n02089867_90.jpg"))

tf.Tensor(1, shape=(), dtype=int32)


In [18]:
preprocessor.label

'n02085620-Chihuahua'

In [ ]:
image_batch, label_batch = next(iter(train_ds))

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].numpy().astype("uint8"))
  label = label_batch[i]
  plt.title(class_names[label])
  plt.axis("off")